#### **Short-Term Wind Power Forecasting using LSTM Recurrent Neural Networks**

#### **Purpose:** : This project implements a machine learning model for predictive modelling of wind power generation using Long Short-Term Memory (LSTM) networks. By processing hourly meteorological data, the model forecasts turbine output to support grid stability and efficient energy dispatch.


##### **Author:** Bello Oluwatobi

##### **Last Updated:** January 20, 2026

### #1 Installing Libraries

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import os

### #2 Loading Dataset

In [2]:
# loading the dataset
df = pd.read_csv('../data/Location1.csv')
df['Timestamp'] = pd.to_datetime(df['Time'])
df = df.set_index('Timestamp')


### #2 Data Exploration



In [3]:
# visualizing the power curve
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df[:1000], x='windspeed_100m', y='Power', alpha=0.3, color='teal')
plt.title('Empirical Power Curve')
plt.xlabel('Wind Speed at 100m (m/s)')
plt.ylabel('Normalized Power Output')
plt.savefig('../results/power_curve.png')
plt.show()

In [ ]:
# visualizing wind speed at different altitudes
plt.figure(figsize=(10, 6))
plt.plot(df['windspeed_10m'].iloc[:100], label='10m Speed')
plt.plot(df['windspeed_100m'].iloc[:100], label='100m Speed (Hub Height)')
plt.title('Wind Speed at Different Altitudes (100m vs 10m)')
plt.xlabel('Time')
plt.ylabel('Wind Speed (m/s)')
plt.legend()
plt.savefig('../results/wind_speed_comparison.png')
plt.show()


### #3 Data Preprocessing & Feature Engineering

In [ ]:
# creating wind direction features in sine and cosine components
df['wind_sin_100m'] = np.sin(df['winddirection_100m'] * np.pi / 180)
df['wind_cos_100m'] = np.cos(df['winddirection_100m'] * np.pi / 180)

In [ ]:
# selecting relevant features and dropping rows with missing values
features = ['windspeed_100m', 'windspeed_10m', 'wind_sin_100m', 'wind_cos_100m', 'temperature_2m', 'Power']
data = df[features].copy().dropna()

In [ ]:
# scaling the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

### #4 Time-Series Sequencing


In [ ]:
# preparing the 24-hour lookback sequences for the LSTM model
lookback = 24
X, y = [], []

for i in range(len(scaled_data) - lookback):
    X.append(scaled_data[i : i + lookback, :])
    y.append(scaled_data[i + lookback, -1])

X, y = np.array(X), np.array(y)

In [ ]:
# splitting into training and testing sets (80-20 split)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


### #5 Building LSTM Model

In [ ]:
# building the sequential LSTM model
model = Sequential([
    
    Input(shape=(X_train.shape[1], X_train.shape[2])),

    LSTM(64, activation='tanh', return_sequences=True),
    Dropout(0.2),

    LSTM(32, activation='tanh'),
    Dense(1)
])

# compiling the model and setting the metrics
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# setting early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.summary()

### #6 Model Training and Evaluation

In [ ]:
# training the model
history = model.fit(
    X_train, y_train,
    epochs=50,        
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

### #7  Model Evaluation


In [ ]:
# making predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
# evaluating the model's performance
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Root Mean Square Error: {rmse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

### #7  Visualizing Model Performance

In [ ]:
# plotting training and validation loss over epochs
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss (MSE)', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)', color='orange')

plt.title('Model Convergence: Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.savefig('../results/model_convergence.png')
plt.show()

In [ ]:
# plotting forecast vs actual for a 15-day period (360 hours)
plt.figure(figsize=(12, 6))
plt.plot(y_test[:360], label='Actual Power', color='black', alpha=0.7)
plt.plot(y_pred[:360], label='LSTM Forecast', color='red', linestyle='--')
plt.title('Wind Power (Forecast vs Actual) for 15-day period')
plt.xlabel('Hours')
plt.ylabel('Normalized Power')
plt.legend()
plt.savefig('../results/forecast_vs_actual.png')
plt.show()



### #8  Saving Model Assets

In [ ]:
# creating directory for model assets
if not os.path.exists('model_assets'):
    os.makedirs('model_assets')

# storing the test dataset
split = int(0.8 * len(X))
test_df_raw = data.iloc[split:].copy()
test_df_raw.reset_index(drop=True, inplace=True)
test_df_raw.to_csv('model_assets/test_data.csv', index=False)

In [ ]:
# saving the model, scaler, and feature columns
model.save('../model_assets/wind_lstm_model.keras')
joblib.dump(scaler, '../model_assets/scaler.gz')
joblib.dump(features, '../model_assets/feature_cols.pkl')